<a href="https://colab.research.google.com/github/azzindani/01_Data_Analytic/blob/main/World_Food_Production_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00 Install Module

In [1]:
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.975s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


## 01 Create Dashboard

In [2]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

# 00 CREATING TAB TITLE

st.set_page_config(
  page_title = 'Food Production',
  page_icon = ':leaves:',
  layout = 'wide',
)


# 01 CREATING DASHBOARD TITLE

st.title(':leaves: World Food Production Dashboard')
st.markdown('<style>div,block-container{padding-top:0rem;}<style>', unsafe_allow_html = True)


# 02 IMPORTING DATASET

dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/World_Food_Production.csv'
df = pd.read_csv(dataset_path)
df = df.rename(columns = {'Entity' : 'Country'})


gdf = gpd.read_file('https://raw.githubusercontent.com/azzindani/00_Data_Source/main/Countries_Geojson.geojson')
gdf = gdf.rename(columns = {'admin' : 'Country'})
gdf = gdf[['Country', 'adm0_a3', 'geometry']]

for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y})

for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass

drop = [
    'Africa',
    'Africa (Fao)',
    'Americas (Fao)',
    'Antigua And Barbuda',
    'Asia',
    'Asia (Fao)',
    'Bahamas',
    'Belgium-Luxembourg (Fao)',
    'Bosnia And Herzegovina',
    'Caribbean (Fao)',
    'Central America (Fao)',
    'Central Asia (Fao)',
    'China (Fao)',
    'Congo',
    "Cote D'Ivoire",
    'Czechia',
    'Czechoslovakia',
    'Democratic Republic Of Congo',
    'Eastern Africa (Fao)',
    'Eastern Asia (Fao)',
    'Eastern Europe (Fao)',
    'Eswatini',
    'Ethiopia (Former)',
    'Europe',
    'Europe (Fao)',
    'European Union (27)',
    'European Union (27) (Fao)',
    'French Guiana',
    'Guadeloupe',
    'Guinea-Bissau',
    'High-Income Countries',
    'Land Locked Developing Countries (Fao)',
    'Least Developed Countries (Fao)',
    'Low Income Food Deficit Countries (Fao)',
    'Low-Income Countries',
    'Lower-Middle-Income Countries',
    'Melanesia',
    'Micronesia (Fao)',
    'Micronesia (Country)',
    'Middle Africa (Fao)',
    'Net Food Importing Developing Countries (Fao)',
    'North America',
    'North Macedonia',
    'Northern Africa (Fao)',
    'Northern America (Fao)',
    'Northern Europe (Fao)',
    'Oceania',
    'Oceania (Fao)',
    'Reunion',
    'Saint Vincent And The Grenadines',
    'Sao Tome And Principe',
    'Serbia',
    'Serbia And Montenegro',
    'Small Island Developing States (Fao)',
    'South America',
    'South America (Fao)',
    'South-Eastern Asia (Fao)',
    'Southern Africa (Fao)',
    'Southern Asia (Fao)',
    'Southern Europe (Fao)',
    'Sudan (Former)',
    'Tanzania',
    'Trinidad And Tobago',
    'Ussr',
    'United States',
    'Upper-Middle-Income Countries',
    'Western Africa (Fao)',
    'Western Asia (Fao)',
    'Western Europe (Fao)',
    'World',
    'Yugoslavia',
]

for x in drop:
  df = df.drop(df[df['Country'] == x].index)


# 03 SETUP TEMPLATE & THEME

colors_1 = px.colors.sequential.Purpor
colors_2 = px.colors.sequential.Purpor_r
explode = tuple([0.015] * 50)
latitude = 20
longitude = 0
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'Purples'
title_x = 0
title_font_size = 18


# 04 CREATING DATE PICKER
col_1, col_2 = st.columns((1, 1))

# Getting min & max date
start_date = df['Year'].min()
end_date = df['Year'].max()

with col_1:
  slider_range = st.slider(
    'Year Slider',
    min_value = start_date,
    max_value = end_date,
    value = [start_date, end_date],
  )

df = df[(df['Year'] >= slider_range[0]) & (df['Year'] <= slider_range[1])] #'''


# 05 CREATING FILTER

var_nums = [
  'Maize Production (Tonnes)',
  'Rice  Production ( Tonnes)',
  'Yams  Production (Tonnes)',
  'Wheat Production (Tonnes)',
  'Tomatoes Production (Tonnes)',
  'Tea  Production ( Tonnes )',
  'Sweet Potatoes  Production (Tonnes)',
  'Sunflower Seed  Production (Tonnes)',
  'Sugar Cane Production (Tonnes)',
  'Soybeans  Production (Tonnes)',
  'Rye  Production (Tonnes)',
  'Potatoes  Production (Tonnes)',
  'Oranges  Production (Tonnes)',
  'Peas, Dry Production ( Tonnes)',
  'Palm Oil  Production (Tonnes)',
  'Grapes  Production (Tonnes)',
  'Coffee, Green Production ( Tonnes)',
  'Cocoa Beans Production (Tonnes)',
  'Meat, Chicken  Production (Tonnes)',
  'Bananas  Production ( Tonnes)',
  'Avocados Production (Tonnes)',
  'Apples Production (Tonnes)',
]

with col_2:
  var_num = st.selectbox('Select Variable', var_nums)

df = df[['Country', 'Year', var_num]]

# 07 CREATING DASHBOARD

# create highlighted indicator

col_11, col_12 = st.columns((1, 2))

chart_df_1 = df.groupby(by = ['Country'], as_index = False,)[[var_num]].sum()
chart_df_1 = chart_df_1.sort_values(by = var_num, ascending = False)[:10]

with col_11:
  title = '10 Highest ' + var_num + ' Country'
  #st.subheader(title)
  fig = px.bar(
    chart_df_1,
    x = var_num,
    y = 'Country',
    template = chart_theme,
    color_discrete_sequence = colors_2,
    title = title,
    text_auto = ',.0f',
  )
  fig.update_layout(
      height = 600,
      margin = margin,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  fig.update_yaxes(categoryorder = 'total ascending')
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_1.style.background_gradient(cmap = cmap))
    csv = chart_df_1.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

var_label = 'Country'
var_number = var_num

map_df = df.groupby(['Country', 'Year'])[var_num].sum()
map_df = map_df.reset_index()
map_df = map_df.fillna(0)

sel_year = map_df['Year'].unique()[-1:]

map_df = gdf.merge(map_df, on = 'Country')
map_df = map_df[(map_df['Year'] >= sel_year.min()) & (map_df['Year'] <= sel_year.max())]

map_df = map_df.set_index(var_label)

with col_12:
  title = 'Latest ' + var_num + ' Geospatial Data'
  fig = px.choropleth_mapbox(
      data_frame = map_df,
      geojson = map_df.geometry,
      locations = map_df.index,
      color = var_number,
      color_continuous_scale = colors_1,
      #range_color = (0, 10),
      opacity = 0.5,
      center = {'lat' : latitude, 'lon' : longitude},
      mapbox_style = 'carto-positron',
      zoom = 1,
      animation_frame = 'Year',
  )

  fig.update_geos(
      #fitbounds = 'locations',
      visible = False,
  )
  fig.update_layout(
      height = 600,
      margin = margin,
      title = title,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

country = st.multiselect('Select Country', df['Country'].unique())

if country:
  filtered_df = df[df['Country'].isin(country)]
else:
  filtered_df = df

#df['Year'] = df['Year'].dt.to_period('Y')
linechart = pd.DataFrame(filtered_df.groupby(['Year'])[[var_num]].sum()).reset_index()
linechart['Year'] = linechart['Year'].astype(str)

# create time series chart

title = 'Yearly ' + var_num
#st.subheader(title)
fig_1 = go.Figure()
fig_1.add_trace(go.Scatter(
  x = linechart['Year'],
  y = linechart[var_num],
  mode = 'lines+markers',
  marker = {'color': colors_2[0]},
))
fig_1.update_traces(
  #hovertemplate = '{:,.0f}'
)
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
  yaxis = dict(tickformat = ',.0f'),
)

st.plotly_chart(fig_1, use_container_width = True, theme = streamlit_theme)

with st.expander('View Data'):
  st.write(linechart.style.background_gradient(cmap = cmap))
  csv = linechart.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

# Download original sample dataset

with st.expander('Sample Data'):
  st.write(df.iloc[:500].style.background_gradient(cmap = cmap))
  csv = df.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = 'Data.csv', mime = 'text/csv')

Overwriting app.py


## 02 Run Dashboard

In [3]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
!npx localtunnel --port 8501

104.198.174.246
npx: installed 22 in 4.469s
your url is: https://orange-melons-listen.loca.lt
^C
